In [1]:
"""
Created on Mon May 13 11:45:31 2024

The goal of this script is to use one dicom file to map a region to will then be used by medtool to create mesh

@author: cofo
"""
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd

import sys
# Add the directory to the sys.path
sys.path.append('D:/EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
%matplotlib qt


In [5]:

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
gr_parent_dir = os.path.dirname(parent_dir)
gr_gr_parent_dir = os.path.dirname(gr_parent_dir)

print(current_dir)
print(parent_dir)
print(gr_parent_dir)
print(gr_gr_parent_dir)


input_file_path = gr_gr_parent_dir + "\\02 - Images Processing\\023 - filtering\\filtered_image-set1.mhd"
output_folder_path = current_dir
meshing_region_path = current_dir + "/map_whole_meshing_region.mhd"

print(input_file_path)
print(output_folder_path)




d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume\python
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\02 - Images Processing\023 - filtering\filtered_image-set1.mhd
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume\python


In [11]:
# Load the image from file
image = sitk.ReadImage(input_file_path)
smoothed_image = sitk.SmoothingRecursiveGaussian(image, sigma=0.3)
meshing_region = sitk.ReadImage(meshing_region_path)

# Convert the SimpleITK image to a NumPy array
image_array = sitk.GetArrayFromImage(image)
image_array_smoothed = sitk.GetArrayFromImage(smoothed_image)

#see it
CT_visualization_functions.browse_images(image)
CT_visualization_functions.browse_images(smoothed_image)

 =================================================================================================================
 ============================================ Defining the left horn =============================================
 =================================================================================================================

In [12]:
# Coordinates of the specific pixel
x, y, z = 369,88,1587

#thresholding
binary_image = CT_modification_functions.segment_from_threshold(smoothed_image,400,1000)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.connected_filter(x, y, z, binary_image)

#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.erode_filter(binary_image,1)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.erode_filter(binary_image,1)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.connected_filter(x, y, z, binary_image)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.dilate_filter(binary_image,2)
#CT_visualization_functions.browse_images(binary_image)

CT_visualization_functions.browse_blended_images(binary_image,meshing_region,True)


In [10]:
## use this cell to create a mask using an interactive window. The mask is then saved as a csv file for repetability

out = CT_visualization_functions.select_pixels(binary_image)
np.savetxt(output_folder_path + '/mask_lefthorn.txt', out, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [14]:
## load the mask from the csv file saved above and apply it to your images

manual_mask_path = output_folder_path + '/mask_lefthorn.txt'
left_horn = sitk.Image(binary_image)

def load_data_as_numpy_array(file_path):
    # Read the data from the file
    data = pd.read_csv(file_path, sep=',', header=0)
    # Convert the list of tuples into a numpy array
    return np.array(data)


#remove selected pixels
manual_mask_array = load_data_as_numpy_array(manual_mask_path)
left_horn_array = sitk.GetArrayFromImage(left_horn)

for row in manual_mask_array:
    left_horn_array[row[2],row[1],row[0]] = 0 #numpy array is a pain so numbers are reversed
    
left_horn = sitk.GetImageFromArray(left_horn_array)

#use connected filter
x, y, z = 463,78,1595
left_horn = CT_modification_functions.connected_filter(x, y, z, left_horn)

left_horn = CT_modification_functions.dilate_filter(left_horn,1)

#visualize mask of left horn
CT_visualization_functions.browse_blended_images(left_horn,meshing_region,True)



D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:198: RuntimeWarning: invalid value encountered in divide
  image1 = image1 / np.max(image1) * 255
D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:203: RuntimeWarning: invalid value encountered in cast
  blended_image = np.clip(blended_image, 0, 255).astype(np.uint8)


 ==================================================================================================================
 ============================================ Defining the right horn =============================================
 ==================================================================================================================

In [15]:
binary_image = CT_modification_functions.segment_from_threshold(smoothed_image,400,1000)

#CT_visualization_functions.browse_images(binary_image)

# Coordinates of the specific pixel
x, y, z = 650,80,1595

binary_image = CT_modification_functions.connected_filter(x, y, z, binary_image)

#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.erode_filter(binary_image,1)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.erode_filter(binary_image,1)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.connected_filter(x, y, z, binary_image)
#CT_visualization_functions.browse_images(binary_image)

binary_image = CT_modification_functions.dilate_filter(binary_image,2)
#CT_visualization_functions.browse_images(binary_image)

CT_visualization_functions.browse_blended_images(binary_image,meshing_region,True)

D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:198: RuntimeWarning: invalid value encountered in divide
  image1 = image1 / np.max(image1) * 255
D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:203: RuntimeWarning: invalid value encountered in cast
  blended_image = np.clip(blended_image, 0, 255).astype(np.uint8)


In [14]:
## use this cell to create a mask using an interactive window. The mask is then saved as a csv file for repetability

out = CT_visualization_functions.select_pixels(binary_image)
np.savetxt(output_folder_path + '/mask_righthorn.txt', out, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [16]:
## load the mask from the csv file saved above and apply it to your images

manual_mask_path = output_folder_path + '/mask_righthorn.txt'
right_horn = sitk.Image(binary_image)

def load_data_as_numpy_array(file_path):
    # Read the data from the file
    data = pd.read_csv(file_path, sep=',', header=0)
    # Convert the list of tuples into a numpy array
    return np.array(data)


#remove selected pixels
manual_mask_array = load_data_as_numpy_array(manual_mask_path)
right_horn_array = sitk.GetArrayFromImage(right_horn)

for row in manual_mask_array:
    right_horn_array[row[2],row[1],row[0]] = 0 #numpy array is a pain so numbers are reversed
    
right_horn = sitk.GetImageFromArray(right_horn_array)

#use connected filter
x, y, z = 650,80,1595
right_horn = CT_modification_functions.connected_filter(x, y, z, right_horn)

right_horn = CT_modification_functions.dilate_filter(right_horn,1)

#visualize mask of left horn
CT_visualization_functions.browse_blended_images(right_horn,image,True)

D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:198: RuntimeWarning: invalid value encountered in divide
  image1 = image1 / np.max(image1) * 255
D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:203: RuntimeWarning: invalid value encountered in cast
  blended_image = np.clip(blended_image, 0, 255).astype(np.uint8)


Adding horn and saving results

In [17]:
result_image = sitk.Add(right_horn, left_horn)
CT_visualization_functions.browse_blended_images(result_image,meshing_region,True)

In [18]:

sitk.WriteImage(result_image, output_folder_path + "/map_keratine_region.mhd")
print(output_folder_path)

d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume\python
